In [ ]:
# TODO: make a dataset for the ROSHAMBO17 dataset (for aedat data as well as AVI frames)
# TODO: make regular time surface
# TODO: make HATS time surface
# TODO: make HOTS time surface
# TODO: train the MCVD model on the three options, as well as the avi frames


# https://ieeexplore.ieee.org/document/8050403
# test loading the rock-paper-scissors dataset
# import tonic
# events = tonic.io.read_aedat("/home/matt/DATA/ROSHAMBO17/recordings/aedat/paper_ale_front.aedat")

def inspect_aedat_file(filename):
    with open(filename, 'rb') as f:
        # Read first 1000 bytes to inspect header
        initial_bytes = f.read(1000)
        print("First 100 bytes as hex:")
        print(initial_bytes[:100].hex())
        print("\nFirst 100 bytes as ASCII (if possible):")
        print(initial_bytes[:100])
        print("\nTotal file size:", f.seek(0, 2))
        f.seek(0)
        print("\nFirst few lines:")
        for i in range(5):
            print(f.readline())

# inspect_aedat_file("/home/matt/DATA/ROSHAMBO17/recordings/aedat/paper_ale_front.aedat")

import numpy as np
import struct

def read_aedat2(filename):
    """
    Read AEDAT2 file format
    Returns: numpy array of events with columns (timestamp, x, y, polarity)
    """
    with open(filename, 'rb') as f:
        # Skip header (look for first non-# line)
        while True:
            line = f.readline()
            if not line.startswith(b'#'):
                break
        
        # Go back 1 line since we read the first data byte
        f.seek(-len(line), 1)
        
        # Read the rest as binary data
        data = f.read()
        
        # Convert bytes to array of uint32
        events = np.frombuffer(data, dtype=np.uint32)
        
        # Reshape to (N, 2) array - pairs of (address, timestamp)
        events = events.reshape(-1, 2)
        
        # Extract address and timestamp
        addresses = events[:, 0]
        timestamps = events[:, 1]
        
        # For DVS128 camera:
        # Bit 0: Polarity (0: OFF, 1: ON)
        # Bits 1-7: Y address (0-127)
        # Bits 8-14: X address (0-127)
        polarity = addresses & 1
        y = (addresses >> 1) & 0x7F
        x = (addresses >> 8) & 0x7F
        
        return np.column_stack((timestamps, x, y, polarity))


# Example usage:
events = read_aedat2("/home/matt/DATA/ROSHAMBO17/recordings/aedat/paper_ale_front.aedat")
print(f"Loaded {len(events)} events")
print("First few events:")
print(events[:5])

Loaded 114632322 events
First few events:
[[2019254326          3          6          0]
 [2019254326         72          4          1]
 [2019254326         66          4          1]
 [2019254326         64          4          1]
 [2019254326         64          4          1]]
